In [46]:
from qiskit import QuantumCircuit
#from qiskit.visualization import plot_histogram
from qiskit.tools.monitor import job_monitor
from azure.quantum.qiskit import AzureQuantumProvider
#from qiskit.tools.visualization import circuit_drawer

import numpy as np
import random

In [47]:
from azure.quantum import Workspace
workspace = Workspace (
    subscription_id = "b1d7f7f8-743f-458e-b3a0-3e09734d716d",
    resource_group = "aq-hackathons",
    name = "aq-hackathon-01",
    location = "eastus"
)

In [48]:
from azure.quantum.qiskit import AzureQuantumProvider
provider = AzureQuantumProvider (
    resource_id = "/subscriptions/b1d7f7f8-743f-458e-b3a0-3e09734d716d/resourceGroups/aq-hackathons/providers/Microsoft.Quantum/Workspaces/aq-hackathon-01",
    location = "eastus"
)

In [49]:
def random_sequence(stages):
    option_1 = np.random.randint(len(gates), size=stages)
    option_2 = np.random.randint(len(gates), size=stages)
    option_2[option_2==option_1] = np.random.randint(len(gates), size=len(option_2[option_2==option_1]))
    return option_1,option_2

def gate_sequences( sequence ):
    gate_sequence = [ number2gate[k] for k in sequence ] #[ gates_dic[number2gate_dic[k]] for k in sequence ]
    return gate_sequence

def qubits4gate( gate ):
    if gate in one_qubit_gates:
        return np.random.randint(number_qubits, size=1)
    else:
        return random.sample(range(number_qubits), 2) #this guarantees different qubits for the control gates

def entanglement_proxi(probs):
    aux = np.ones(int(np.power(2,number_qubits)))/np.power(2,number_qubits)

    for i,key in enumerate(probs):
        #print(key,probs[key],i)
        aux[i] = aux[i]-probs[key]
    aux = np.sum(np.absolute(aux))/1.875
    score = 100*(1-aux)
    return score 

In [50]:
stages = 5
number_qubits = 4
number_bits = 3

In [51]:
max_score = 0
winner = 'No one'
def play_game(max_score, winner, backend='simulator'):

    circuit = QuantumCircuit(number_qubits,number_qubits)

    gates = {'X': circuit.x, 
            'Y': circuit.y, 
            'Z': circuit.z, 
            'Hadamard': circuit.h,
            'CNOT': circuit.cnot, 
            'Swap': circuit.swap, 
            'CHadamard': circuit.ch,
            'CX':circuit.cx,
            'CY':circuit.cy,
            'CZ':circuit.cz}

    number2gate = {k:key for (k,key) in zip(range(0,len(gates)),list(gates.keys()))}

    one_qubit_gates = ['X', 'Y', 'Z', 'Hadamard']

    option1,option2 = random_sequence(stages)
    option1 = gate_sequences(option1)
    option2 = gate_sequences(option2)

    qubits_option1 = [qubits4gate(gate) for gate in option1]
    qubits_option2 = [qubits4gate(gate) for gate in option2]

    choices = {'a':[option1,qubits_option1] , 'b':[option2,qubits_option2]}

    chosen_gates = []
    arguments = []
    measurement = [i for i in range(number_qubits)]

    print('Welcome to the Amaze Maze. The goal of the game is to entangle your qubits as much as possible!')
    print('Remember you start in the qubit |0000>')
    for i in range(stages):
        print("This is stage {0} out of {1}".format(i+1, stages))
        print('Your options are:')
        print('Apply a) the {0} gate on qubit(s): {1} or b) the {2} gate on qubit(s): {3}'.format(option1[i], qubits_option1[i],option2[i],qubits_option2[i]))
        choice = input('Choose a or b: ')
        chosen_gates.append(choices[choice][0][i])
        arguments.append(choices[choice][1][i])

        gate = gates[chosen_gates[i]]
        if chosen_gates[i] in one_qubit_gates:
            q1 = arguments[i][0]
            gate(q1)
        else:
            q1 = arguments[i][0]
            q2 = arguments[i][1]
            gate(q1,q2)

    circuit.measure(measurement, measurement)
    
    if backend=='simulator':
        simulator_backend = provider.get_backend("ionq.simulator")
        job = simulator_backend.run(circuit, shots=100)

    if backend=='qpu':
        qpu_backend = provider.get_backend("ionq.qpu")
        job = qpu_backend.run(circuit, shots=1024)

    job_monitor(job)
    result = job.result()
    #print(result)

    probs = result.data()['probabilities']
    score = entanglement_proxi(probs)
    print('Your score is: {0}'.format(score))
    if score > max_score: 
        max_score = score
        winner = input('Write your name: ')
    print('The record is: {0} and its held by {1}'.format(score, winner))
    return(max_score, winner, circuit)

In [52]:
max_score, winner, circuit = play_game(max_score, winner, 'qpu')
print('Your quantum circuit through the maze is: ')
circuit.draw()

Welcome to the Amaze Maze. The goal of the game is to entangle your qubits as much as possible!
Remember you start in the qubit |0000>
This is stage 1 out of 5
Your options are:
Apply a) the CX gate on qubit(s): [3, 0] or b) the CHadamard gate on qubit(s): [0, 3]
This is stage 2 out of 5
Your options are:
Apply a) the Y gate on qubit(s): [0] or b) the CZ gate on qubit(s): [2, 0]
This is stage 3 out of 5
Your options are:
Apply a) the CNOT gate on qubit(s): [0, 2] or b) the CX gate on qubit(s): [3, 1]
This is stage 4 out of 5
Your options are:
Apply a) the CX gate on qubit(s): [1, 0] or b) the Swap gate on qubit(s): [1, 0]
This is stage 5 out of 5
Your options are:
Apply a) the CY gate on qubit(s): [3, 1] or b) the Z gate on qubit(s): [0]
Job Status: job has successfully run
Your score is: 0.0864583333333333
The record is: 0.0864583333333333 and its held by Braulio
Your quantum circuit through the maze is: 


┌───┐┌───┐     ┌───┐┌───┐┌─┐
q_0: ┤ X ├┤ Y ├─────┤ X ├┤ Z ├┤M├
     └─┬─┘└───┘┌───┐└─┬─┘└┬─┬┘└╥┘
q_1: ──┼───────┤ X ├──■───┤M├──╫─
       │   ┌─┐ └─┬─┘      └╥┘  ║ 
q_2: ──┼───┤M├───┼─────────╫───╫─
       │   └╥┘   │   ┌─┐   ║   ║ 
q_3: ──■────╫────■───┤M├───╫───╫─
            ║        └╥┘   ║   ║ 
c: 4/═══════╩═════════╩════╩═══╩═
            2         3    1   0